In [1]:
%run ml_header.py

In [2]:
%run get_data.py
benchmarks = ('adpcm_encoder', 'aes', 'ann', 'average', 'decimation', 'fft_fixed', 'fir',
              'idct', 'interpolation', 'kasumi', 'qsort', 'snow3g', 'sobel')  # 13
gd = GetData('data/ASIC-2-FPGA', benchmarks)
gd.main()

In [3]:
%run methods.py

In [4]:
%run machine_learning.py

In [5]:
import itertools

In [6]:
features = ['AREA', 'state', 'FU', 'REG', 'MUX', 'DEC', 'pin_pair',
            'net', 'max', 'min', 'ave', 'MISC', 'MEM', 'sim', 'Pmax',
            'Pmin', 'Pave', 'Latency', 'BlockMemoryBit', 'DSP', 'Slices']
invalid_features = ['Slices', 'Latency']
valid_features = [i for i in features if i not in invalid_features]
label = 'Slices'

In [7]:
def execute(df, estimator, kind):
    # remove attribute columns
    data = df[features].copy()
    # fix missing data
    data = ML.fix_missing_data(data)
    # X, y
    X, y = ML.separate_feature_label(data, valid_features=valid_features, label=label)
    # feature scaling
    X = ML.feature_scaling(X)
    if kind == 'training':
        estimator.fit(X, y)
        return estimator
    elif kind == 'testing':
        y_pred = estimator.predict(X)
        data['AREA'] = y_pred
        return DirectMapping.main(data)

In [14]:
def train_es(data=None, estimator=None, result_fn=None):
    scores_all = dict()
    for benchmarks_cnt in range(1, len(benchmarks)):
        scores = dict()
        for benchmarks_train in itertools.combinations(benchmarks, benchmarks_cnt):
            benchmarks_test = (i for i in benchmarks if i not in benchmarks_train)
            data_train = pd.concat([data[i] for i in benchmarks_train], axis=0, ignore_index=True)
            estimator = execute(data_train, estimator, kind='training')
            key = ';'.join(list(benchmarks_train))
            scores[key] = dict()
            for benchmark_test in benchmarks_test:
                scores[key][benchmark_test] = execute(data[benchmark_test], estimator, kind='testing')
        scores_all[str(benchmarks_cnt)] = scores
        
    with open(result_fn, 'w') as f:
        f.write('train,test,adrs_ave,adrs_max,adrs_ave_rms,adrs_max_rms,hypervolume,dominance,cardinality\n')
        for bench_cnt in scores_all.keys():
            for training, testing in scores_all[bench_cnt].items():
                for k, v in testing.items():
                    f.write(training + ',' + k + ',' + ','.join([f'{i:.4f}' for i in v.values()]) + '\n')
                    
    return scores_all

In [15]:
%%time
scores_v4_lr = train_es(data=gd.data_v4, estimator=estimators[0], result_fn='es_train_lr.csv')

Wall time: 1h 59min 36s


In [21]:
%%time
scores_v4_ada = train_es(data=gd.data_v4, estimator=estimators[-2], result_fn='es_v4_ada.csv')

Wall time: 16h 55min 3s
